## **TD 11-12**

### Collecte, Traitement, et Analyse de données de réseaux sociaux

LEFEVRE Laura et LE CORRE Camille - LDD BI

In [51]:
#Importation des modules
import json
import re
from textblob import TextBlob
import textblob_fr

#Module a installer
    #!pip install textblob
    #!pip install textblob-fr

In [86]:
def openFile(fileName):
    with open(fileName, encoding='utf8') as json_data:
        data_dict = json.load(json_data)
        return data_dict

In [179]:
dic = openFile("versailles_tweets_100.json")
dic[2]


{'_id': '1421599163561742339',
 'public_metrics': {'retweet_count': 0,
  'reply_count': 1,
  'like_count': 1,
  'quote_count': 0},
 'id': '1421599163561742339',
 'conversation_id': '1421599163561742339',
 'context_annotations': [{'domain': {'id': '6', 'name': 'Sports Event'},
   'entity': {'id': '1103026158382141440',
    'name': 'Tokyo 2020 Summer Olympics',
    'description': 'Tokyo 2020 Summer Olympics '}}],
 'author_id': '1339914264522461187',
 'text': 'Ah oui le sommeil là sera compliqué. #CIV  est éliminé des JO , Ahi on peut faire ça ?',
 'geo': {'place_id': '00b8943291443c8c'},
 'lang': 'fr',
 'created_at': '2021-07-31T22:30:27.000Z',
 'entities': {'hashtags': [{'start': 37, 'end': 41, 'tag': 'CIV'}]}}

In [159]:
dic[0]['text']

'Goumin des éléphants joueurs la même fatigue même 😫 #twitter225'

In [160]:
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

In [165]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[A-Za-z\.]*[0-9]+[A-Za-z%°\.]*", "", text)
    text = re.sub(r"[\,\!\?\%\(\)\/\"\&\+\#\$\£\%\:\@\-]", "", text)
    text = re.sub(r"[\é\è\ê]", "e", text)
    text = re.sub(r"[\ù]", "u", text)
    text = re.sub(r"[\à]", "a", text)
    text = re.sub(r"[\î\ï]", "i", text)
    text = remove_emojis(text)
    text = re.sub(r"( )+", " ", text)
    text = re.sub(r"( )*$", "", text)
    
    return text

In [164]:
clean_text(dic[0]['text'])

'goumin des elephants joueurs la meme fatigue meme'

In [191]:
def ZoneAtterrissage(tweet):
    tweet_clean = clean_text(tweet['text'])
    tweet['text'] = tweet_clean
    with open("zone_atterrissage.json", "w") as filout:
        json.dump(tweet, filout)


In [189]:
def traitement(liste_tweet):
    
    for elt in liste_tweet:
        ZoneAtterrissage(elt)

In [190]:
traitement(dic)

In [46]:
testimonial = TextBlob("Textblob is amazingly simple to use. What great fun!")
testimonial.sentiment

Sentiment(polarity=0.39166666666666666, subjectivity=0.4357142857142857)